# FEATURE SELECTION

This notebooks implements the XGBoost wrapper method for feature selection. The Random Forest method had been implemented as well but has been discarded due to poor results and low stability.

In [0]:
import os
import numpy as np
import pandas as pd
import pickle

import xgboost

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import ndcg_score

from google.colab import drive

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
basepath = '/content/drive/My Drive/AML - AIRBNB/'

In [0]:
X_train = pd.read_pickle(os.path.join(basepath, 'X_train_v2.pkl'))

In [0]:
y_train = pd.read_pickle(os.path.join(basepath, 'y_train_v2.pkl'))

In [0]:
X_val = pd.read_pickle(os.path.join(basepath, 'X_val.pkl'))
y_val = pd.read_pickle(os.path.join(basepath, 'y_val.pkl'))

In [0]:
params = {'eta': 0.09,
          'max_depth': 6,
          'subsample': 0.5,
          'colsample_bytree': 0.5,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 12,
          'tree_method': 'gpu_hist',
          'missing': -1}
num_rounds = 1000
xg_train = xgboost.DMatrix(X_train, label=y_train)  
xg_valid = xgboost.DMatrix(X_val, label=y_val)  
xgb = xgboost.train(params, xg_train, num_rounds, [(xg_train,'train'), 
                                                   (xg_valid, 'validation')],
                    early_stopping_rounds=10)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[0]	train-mlogloss:2.35089	validation-mlogloss:2.31248
Multiple eval metrics have been passed: 'validation-mlogloss' will be used for early stopping.

Will train until validation-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:2.25493	validation-mlogloss:2.22005
[2]	train-mlogloss:2.14457	validation-mlogloss:2.05037
[3]	train-mlogloss:2.05501	validation-mlogloss:1.9323
[4]	train-mlogloss:1.97858	validation-mlogloss:1.8387
[5]	train-mlogloss:1.92156	validation-mlogloss:1.79539
[6]	train-mlogloss:1.86585	validation-mlogloss:1.72878
[7]	train-mlogloss:1.81507	validation-mlogloss:1.69922
[8]	train-mlogloss:1.77019	validation-mlogloss:1.6603
[9]	train-mlogloss:1.72574	validation-mlogloss:1.60802
[10]	train-mlogloss:1.68412	validation-mlogloss:1.55982
[11]	train-mlogloss:1.6525	validation-mlogloss:1.54208
[12]	train-mlogloss:1.61609	validation-mlogloss:1.51126
[13]	train-mlogloss:1.58078	validation-mlogloss:1.47636
[14]	train-mlogloss:1.55398	validation-mlogloss:1.45853
[15]	train-

In [0]:
scores = pd.DataFrame([xgb.get_fscore()]).squeeze()
scores /= scores.max()

In [0]:
to_keep = len(scores) if len(scores) < X_train.shape[1] // 4 * 3 else X_train.shape[1] // 4 * 3
feat_xgboost = scores.sort_values()[::-1][:to_keep].index.tolist()

In [0]:
with open(os.path.join(basepath, 'xgboost_features.pkl'), 'wb') as f:
  pickle.dump(feat_xgboost, f)

We also score this model against validation data, as a baseline.

In [0]:
preds = xgb.predict(xgboost.DMatrix(X_val))

In [0]:
ndcg_score(LabelBinarizer().fit_transform(y_val), preds)

0.8622626688967371